In [27]:
import json
import re
import time
import requests
import pandas as pd
import numpy as np
import unicodedata
pd.set_option('max_colwidth', None)
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def name_to_keep_ind(groups):
    """
    Function to determine if a text should be kept or not.

    Input:
    groups: list of character groups

    Output:
    0: if text should be not used
    1: if text should be used
    """
    # Groups of characters that do not perform well
    groups_to_skip = ['HIRAGANA', 'CJK', 'KATAKANA','ARABIC', 'HANGUL', 'THAI','DEVANAGARI','BENGALI',
                      'THAANA','GUJARATI','CYRILLIC']
    
    if any(x in groups_to_skip for x in groups):
        return 0
    else:
        return 1

def remove_non_latin_characters(text):
    """
    Function to remove non-latin characters.

    Input:
    text: string of characters

    Output:
    final_char: string of characters with non-latin characters removed
    """
    final_char = []
    groups_to_skip = ['HIRAGANA', 'CJK', 'KATAKANA','ARABIC', 'HANGUL', 'THAI','DEVANAGARI','BENGALI',
                      'THAANA','GUJARATI','CYRILLIC']
    for char in text:
        try:
            script = unicodedata.name(char).split(" ")[0]
            if script not in groups_to_skip:
                final_char.append(char)
        except:
            pass
    return "".join(final_char)
    
def group_non_latin_characters(text):
    """
    Function to group non-latin characters and return the number of latin characters.

    Input:
    text: string of characters

    Output:
    groups: list of character groups
    latin_chars: number of latin characters
    """
    groups = []
    latin_chars = []
    text = text.replace(".", "").replace(" ", "")
    for char in text:
        try:
            script = unicodedata.name(char).split(" ")[0]
            if script == 'LATIN':
                latin_chars.append(script)
            else:
                if script not in groups:
                    groups.append(script)
        except:
            if "UNK" not in groups:
                groups.append("UNK")
    return groups, len(latin_chars)

def check_for_non_latin_characters(text):
    """
    Function to check if non-latin characters are dominant in a text.

    Input:
    text: string of characters

    Output:
    0: if text should be not used
    1: if text should be used
    """
    groups, latin_chars = group_non_latin_characters(str(text))
    if name_to_keep_ind(groups) == 1:
        return 1
    elif latin_chars > 20:
        return 1
    else:
        return 0

In [11]:
def clean_title(old_title):
    """
    Function to check if title should be kept and then remove non-latin characters. Also
    removes some HTML tags from the title.
    
    Input:
    old_title: string of title
    
    Output:
    new_title: string of title with non-latin characters and HTML tags removed
    """
    keep_title = check_for_non_latin_characters(old_title)
    if (keep_title == 1) & isinstance(old_title, str):
        new_title = remove_non_latin_characters(old_title)
        if '<' in new_title:
            new_title = new_title.replace("<i>", "").replace("</i>","")\
                                 .replace("<sub>", "").replace("</sub>","") \
                                 .replace("<sup>", "").replace("</sup>","") \
                                 .replace("<em>", "").replace("</em>","") \
                                 .replace("<b>", "").replace("</b>","") \
                                 .replace("<I>", "").replace("</I>", "") \
                                 .replace("<SUB>", "").replace("</SUB>", "") \
                                 .replace("<scp>", "").replace("</scp>", "") \
                                 .replace("<font>", "").replace("</font>", "") \
                                 .replace("<inf>","").replace("</inf>", "") \
                                 .replace("<i /> ", "") \
                                 .replace("<p>", "").replace("</p>","") \
                                 .replace("<![CDATA[<B>", "").replace("</B>]]>", "") \
                                 .replace("<italic>", "").replace("</italic>","")\
                                 .replace("<title>", "").replace("</title>", "") \
                                 .replace("<br>", "").replace("</br>","").replace("<br/>","") \
                                 .replace("<B>", "").replace("</B>", "") \
                                 .replace("<em>", "").replace("</em>", "") \
                                 .replace("<BR>", "").replace("</BR>", "") \
                                 .replace("<title>", "").replace("</title>", "") \
                                 .replace("<strong>", "").replace("</strong>", "") \
                                 .replace("<formula>", "").replace("</formula>", "") \
                                 .replace("<roman>", "").replace("</roman>", "") \
                                 .replace("<SUP>", "").replace("</SUP>", "") \
                                 .replace("<SSUP>", "").replace("</SSUP>", "") \
                                 .replace("<sc>", "").replace("</sc>", "") \
                                 .replace("<subtitle>", "").replace("</subtitle>", "") \
                                 .replace("<emph/>", "").replace("<emph>", "").replace("</emph>", "") \
                                 .replace("""<p class="Body">""", "") \
                                 .replace("<TITLE>", "").replace("</TITLE>", "") \
                                 .replace("<sub />", "").replace("<sub/>", "") \
                                 .replace("<mi>", "").replace("</mi>", "") \
                                 .replace("<bold>", "").replace("</bold>", "") \
                                 .replace("<mtext>", "").replace("</mtext>", "") \
                                 .replace("<msub>", "").replace("</msub>", "") \
                                 .replace("<mrow>", "").replace("</mrow>", "") \
                                 .replace("</mfenced>", "").replace("</math>", "")

            if '<mml' in new_title:
                all_parts = [x for y in [i.split("mml:math>") for i in new_title.split("<mml:math")] for x in y if x]
                final_parts = []
                for part in all_parts:
                    if re.search(r"\>[$%#!^*\w.,/()+-]*\<", part):
                        pull_out = re.findall(r"\>[$%#!^*\w.,/()+-]*\<", part)
                        final_pieces = []
                        for piece in pull_out:
                            final_pieces.append(piece.replace(">", "").replace("<", ""))
                        
                        final_parts.append(" "+ "".join(final_pieces) + " ")
                    else:
                        final_parts.append(part)
                
                new_title = "".join(final_parts).strip()
            else:
                pass

            if '<xref' in new_title:
                new_title = re.sub(r"\<xref[^/]*\/xref\>", "", new_title)

            if '<inline-formula' in new_title:
                new_title = re.sub(r"\<inline-formula[^/]*\/inline-formula\>", "", new_title)

            if '<title' in new_title:
                new_title = re.sub(r"\<title[^/]*\/title\>", "", new_title)

            if '<p class=' in new_title:
                new_title = re.sub(r"\<p class=[^>]*\>", "", new_title)
            
            if '<span class=' in new_title:
                new_title = re.sub(r"\<span class=[^>]*\>", "", new_title)

            if 'mfenced open' in new_title:
                new_title = re.sub(r"\<mfenced open=[^>]*\>", "", new_title)
            
            if 'math xmlns' in new_title:
                new_title = re.sub(r"\<math xmlns=[^>]*\>", "", new_title)

        if '<' in new_title:
            new_title = new_title.replace(">i<", "").replace(">/i<", "") \
                                 .replace(">b<", "").replace(">/b<", "") \
                                 .replace("<inline-formula>", "").replace("</inline-formula>","")
        if new_title.isupper():
            new_title = new_title.title()
        
        return new_title
    else:
        return ''
    
def clean_abstract(raw_abstract, inverted=False):
    """
    Function to clean abstract and return it in a format for the model.
    
    Input:
    raw_abstract: string of abstract
    inverted: boolean to determine if abstract is inverted index or not
    
    Output:
    final_abstract: string of abstract in format for model
    """
    if inverted:
        if isinstance(raw_abstract, dict) | isinstance(raw_abstract, str):
            if isinstance(raw_abstract, dict):
                invert_abstract = raw_abstract
            else:
                invert_abstract = json.loads(raw_abstract)
            
            if invert_abstract.get('IndexLength'):
                ab_len = invert_abstract['IndexLength']

                if ab_len > 20:
                    abstract = [" "]*ab_len
                    for key, value in invert_abstract['InvertedIndex'].items():
                        for i in value:
                            abstract[i] = key
                    final_abstract = " ".join(abstract)[:2500]
                    keep_abs = check_for_non_latin_characters(final_abstract)
                    if keep_abs == 1:
                        pass
                    else:
                        final_abstract = None
                else:
                    final_abstract = None
            else:
                if len(invert_abstract) > 20:
                    abstract = [" "]*1200
                    for key, value in invert_abstract.items():
                        for i in value:
                            try:
                                abstract[i] = key
                            except:
                                pass
                    final_abstract = " ".join(abstract)[:2500].strip()
                    keep_abs = check_for_non_latin_characters(final_abstract)
                    if keep_abs == 1:
                        pass
                    else:
                        final_abstract = None
                else:
                    final_abstract = None
                
        else:
            final_abstract = None
    else:
        if raw_abstract:
            ab_len = len(raw_abstract)
            if ab_len > 30:
                final_abstract = raw_abstract[:2500]
                keep_abs = check_for_non_latin_characters(final_abstract)
                if keep_abs == 1:
                    pass
                else:
                    final_abstract = None
            else:
                final_abstract = None
        else:
            final_abstract = None
            
    return final_abstract

In [12]:
def get_top_keywords(title, abstract, cand_embs_df):
    """
    Function to use title, abstract, and candidate keyword embeddings to return scores.
    
    Input:
    title: title of paper
    abstract: abstract of paper
    cand_embs_df: dataframe containing keywords and embeddings (filtered by paper topics)
    
    Output:
    final_abstract: string of abstract in format for model
    """
    cand_embs_df = cand_embs_df.copy()
    if title.isupper():
        title = title.title()
    if abstract:
        title_and_abstract = f"{title}\n {abstract}"
    else:
        if title:
            title_and_abstract = f"{title}"
        else:
            title_and_abstract = ""

    if title_and_abstract:
        # Get title/abstract embedding
        title_abs_emb = emb_model.encode(title_and_abstract)
    
        # Get scores for each candidate keyword
        cand_embs_df['cand_scores'] = cand_embs_df['embedding'].apply(lambda x: np.dot(title_abs_emb, x))
    else:
        cand_embs_df['cand_scores'] = -1
    
    return cand_embs_df

In [13]:
def get_candidate_keywords(candidate_topics):
    """
    Function to get keywords based on the topics
    
    Input:
    candidate_topics: topics of paper
    
    Output:
    keywords_data_copy: filtered df of keywords and embeddings
    """
    keywords_data_copy = all_keywords_data[all_keywords_data['topic_id'].isin(candidate_topics)]\
        .drop_duplicates(subset=['keywords'])[['keywords','embedding']].copy()
    return keywords_data_copy

In [14]:
def get_all_keywords(candidate_topics, paper_title, abstract, invert_abstract=False, topk=5):
    """
    Function to get keywords that match title/abstract
    
    Input:
    candidate_topics: topic ids for a paper
    paper_title: title of a paper
    abstract: abstract of a paper
    invert_abstract: whether or not the abstract is being input as an inverted index (True/False)
    topk: maximum number of keywords to pull for a paper
    
    Output:
    final_keywords
    """
    # Process title and abstract
    paper_title = clean_title(paper_title)
    abstract = clean_abstract(abstract, inverted=invert_abstract)
    
    # Get candidate keywords
    keywords_data = get_candidate_keywords(candidate_topics)
    if keywords_data.shape[0]>0:
    
        # Get candidate scores
        cand_scores = get_top_keywords(paper_title, abstract, keywords_data)
        if cand_scores[cand_scores['cand_scores']>=0].shape[0] > 0:
            top_k = cand_scores[cand_scores['cand_scores']>=0].sort_values('cand_scores', ascending=False).head(topk)
            top_k['keywords'] = top_k['keywords'].apply(lambda x: x.lower())
            top_k = top_k.drop_duplicates(subset=['keywords'])
            keywords = top_k['keywords'].tolist()
            scores = top_k['cand_scores'].tolist()
    
            final_keywords = []
            _ = [final_keywords.append({"keyword": keyword, "score": score}) for keyword, score in zip(keywords, scores) if score > 0.50]
    
            if final_keywords:
                return final_keywords
            else:
                if scores[0] > 0.40:
                    return [{"keyword": keywords[0], "score": scores[0]}]
                else:
                    return []
        else:
            return []
    else:
        return []

#### Code for testing (not needed in final predict.py file

In [7]:
def get_paper_id_resp(paper_id):
    open_req = f"https://api.openalex.org/works/W{str(paper_id)}"
    resp = requests.get(open_req)
    if resp.status_code == 200:
        return resp.json()
    else:
        return []

In [8]:
def get_topics_for_paper(paper_id_resp):

    topics = []
    scores = []
    try:
        all_topics = paper_id_resp['topics']
        for i in all_topics:
            topics.append(int(i['id'].split('/T')[1]))
            scores.append(i['score'])
    except:
        pass

    if topics:
        return [x for _, x in sorted(zip(scores, topics), key=lambda pair: pair[0], reverse=True)]
    else:
        return topics

In [33]:
def test_function_to_simulate_data(paper_id):
    paper_id_resp = get_paper_id_resp(paper_id)
    paper_title = paper_id_resp['title']
    abstract = paper_id_resp['abstract_inverted_index']

    
    candidate_topics = get_topics_for_paper(paper_id_resp)
    
    first_time = time.time()
    all_keywords = get_all_keywords(candidate_topics, paper_title, abstract, invert_abstract=True, topk=5)
    print(time.time() - first_time)
    return all_keywords

In [34]:
%%time
for i in [2995104085,2492191525,2780733096]:
    print(test_function_to_simulate_data(i))

0.10874819755554199
[{'keyword': 'high-temperature applications', 'score': 0.5638906796476346}, {'keyword': 'solidification modeling', 'score': 0.5359686474935086}, {'keyword': 'thermochemical', 'score': 0.5137059869882088}]
0.23958754539489746
[{'keyword': 'right-wing extremism', 'score': 0.43486157318306873}]
0.0816946029663086
[{'keyword': 'judicial cooperation', 'score': 0.49244597941582136}]
CPU times: user 3.48 s, sys: 0 ns, total: 3.48 s
Wall time: 661 ms


### Examples straight from OpenAlex

In [17]:
emb_model = SentenceTransformer('baai/BGE-M3')

In [19]:
# topics = pd.read_parquet("topics_for_keyword_pred.parquet")
# topics_explode = topics[['topic_id','keywords']].explode('keywords').copy()
# all_keyword_embs = emb_model.encode(topics_explode['keywords'].tolist())
# topics_explode['embedding'] = [np.array(x) for x in all_keyword_embs.tolist()]
# topics_explode.to_parquet('current_keyword_embs.parquet')
all_keywords_data = pd.read_parquet('s3://openalex-keywords-matcher/v1/keywords_files/')

In [164]:
examples = pd.read_parquet("abstracts_sample_file_single")

In [165]:
examples['abstract_processed'] = examples['abstract'].apply(lambda x: clean_abstract(x, inverted=True))

In [166]:
samples = examples.sample(100)

In [179]:
samples['keywords'] = samples['paper_id'].apply(test_function_to_simulate_data)

In [180]:
samples['keywords_len'] = samples['keywords'].apply(len)

In [181]:
samples.sample(10)

,paper_id,original_title,abstract,abstract_processed,keywords,keywords_len
660,2995104085,Thermodynamic Simulation of Polycrystalline Silicon Chemical Vapor Deposition in Si–Cl–H System,"{""IndexLength"":206,""InvertedIndex"":{""Based"":[0],""on"":[1,91],""thermodynamic"":[2,72],""data"":[3],""for"":[4,35,45,109,121],""related"":[5],""pure"":[6],""substances,"":[7],""the"":[8,18,25,32,79,103,110,126,130,158,181,186,190,193,203],""relations"":[9],""of"":[10,27,74,106,175],""(nCl/nH)Eq"":[11],""and"":[12,29,64,153,170],""(nCl/nH)o"":[13],""have"":[14,161],""been"":[15,83,162],""plotted"":[16],""in"":[17,78,125,185,202],""Si–Cl–H"":[19,127],""system."":[20,128],""The"":[21,85,95,116,142],""results"":[22],""show"":[23],""that"":[24],""difference"":[26],""(nSi/nCl)o"":[28],""(nSi/nCl)Eq"":[30],""is"":[31,43,58,67,98,113,119,197],""driving"":[33],""force"":[34],""polycrystalline"":[36,46,75,92,122],""silicon"":[37,47,76,93,123,131,194],""chemical"":[38],""vapor"":[39],""deposition"":[40,48,132],""(CVD)."":[41],""SiHCl3"":[42,173],""preferred"":[44],""to"":[49,172,180],""SiCl4."":[50],""SiH2Cl2"":[51],""would"":[52],""be"":[53,148],""even"":[54],""better,"":[55],""but"":[56],""it"":[57,66],""not"":[59],""stable"":[60],""as"":[61,164],""a"":[62,88],""gas"":[63],""hence"":[65],""less"":[68],""frequently"":[69],""used."":[70],""Then,"":[71],""simulation"":[73],""CVD"":[77,124],""Si–H–Cl"":[80],""system"":[81],""has"":[82,87],""investigated."":[84],""pressure"":[86],""negative"":[89],""effect"":[90],""yield."":[94],""optimum"":[96,143,159,191],""temperature"":[97],""1400"":[99,165],""K,"":[100,166],""at"":[101],""which"":[102,177],""kinetic"":[104],""rate"":[105,133],""rate-determining"":[107],""step"":[108],""main"":[111],""reaction"":[112],""large"":[114],""enough."":[115],""excess"":[117],""hydrogen"":[118],""necessary"":[120],""However,"":[129],""increases"":[134],""then"":[135],""decreases"":[136],""with"":[137],""increasing"":[138],""H2"":[139,144,171],""molar"":[140,145],""fraction."":[141],""fraction"":[146],""should"":[147],""determined"":[149],""by"":[150],""considering"":[151],""thermodynamics"":[152],""transport"":[154],""phenomena"":[155],""simultaneously."":[156],""Finally,"":[157],""conditions"":[160],""obtained"":[163],""about"":[167],""0.1"":[168],""MPa,"":[169],""ratio"":[174,196],""15,"":[176],""are"":[178],""close"":[179],""limited"":[182],""reported"":[183,201],""values"":[184],""open"":[187,204],""literature."":[188,205],""Under"":[189],""conditions,"":[192],""yield"":[195],""34.82%"":[198],""against"":[199],""20%"":[200]}}","Based on thermodynamic data for related pure substances, the relations of (nCl/nH)Eq and (nCl/nH)o have been plotted in the Si–Cl–H system. The results show that the difference of (nSi/nCl)o and (nSi/nCl)Eq is the driving force for polycrystalline silicon chemical vapor deposition (CVD). SiHCl3 is preferred for polycrystalline silicon deposition to SiCl4. SiH2Cl2 would be even better, but it is not stable as a gas and hence it is less frequently used. Then, thermodynamic simulation of polycrystalline silicon CVD in the Si–H–Cl system has been investigated. The pressure has a negative effect on polycrystalline silicon yield. The optimum temperature is 1400 K, at which the kinetic rate of rate-determining step for the main reaction is large enough. The excess hydrogen is necessary for polycrystalline silicon CVD in the Si–Cl–H system. However, the silicon deposition rate increases then decreases with increasing H2 molar fraction. The optimum H2 molar fraction should be determined by considering thermodynamics and transport phenomena simultaneously. Finally, the optimum conditions have been obtained as 1400 K, about 0.1 MPa, and H2 to SiHCl3 ratio of 15, which are close to the limited reported values in the open literature. Under the optimum conditions, the silicon yield ratio is 34.82% against 20% reported in the open literature.","[{'keyword': 'high-temperature applications', 'score': 0.56389058

In [182]:
samples['keywords_len'].value_counts()

keywords_len
1    40
0    21
2    12
3    10
5     9
4     8
Name: count, dtype: int64

In [183]:
samples[samples['abstract'].isnull()]['keywords_len'].value_counts()

keywords_len
1    18
0    14
2     6
3     5
5     4
4     3
Name: count, dtype: int64

In [184]:
samples[~samples['abstract'].isnull()]['keywords_len'].value_counts()

keywords_len
1    22
0     7
2     6
3     5
4     5
5     5
Name: count, dtype: int64

In [188]:
samples[samples['abstract'].isnull()].sample(10)[['paper_id','original_title','abstract_processed','keywords']]

,paper_id,original_title,abstract_processed,keywords
2220,2793828623,Doce lecciones : comentarios a los acontecimientos de Hungria / [José Miguel de Azaola].,None,"[{'keyword': 'historiography', 'score': 0.4431596018212326}]"
299,2589565674,"Creativity, Exploration and Control in Musical Parameter Spaces.",None,"[{'keyword': 'musical performance', 'score': 0.5948892984307431}, {'keyword': 'sound synthesis', 'score': 0.5936375563721186}, {'keyword': 'music generation', 'score': 0.5902165441787353}, {'keyword': 'digital musical instruments', 'score': 0.558197739792075}, {'keyword': 'acoustic ecology', 'score': 0.556107683092431}]"
1442,2104376796,Eine genaue Mikrobestimmungsmethode für Arsen in biologischem Material,None,"[{'keyword': 'scanning electrochemical microscopy', 'score': 0.5208396966097735}, {'keyword': 'anodic stripping voltammetry', 'score': 0.5188365755426492}, {'keyword': 'detection', 'score': 0.5174081585205104}, {'keyword': 'heavy metal ions', 'score': 0.5082847403643969}]"
503,1541307873,Ocena nierównomierności nacisków w hamulcach wielotarczowych,None,"[{'keyword': 'marine engine diagnostics', 'score': 0.5327456006205008}, {'keyword': 'sensitivity analysis', 'score': 0.5062124892476537}, {'keyword': 'vehicle technical condition', 'score': 0.5035523748246806}]"
2357,1510515863,A study of the overoxidation of the conducting polymer polypyrrole,None,"[{'keyword': 'conducting polymers', 'score': 0.6650822905053988}, {'keyword': 'polyaniline', 'score': 0.5071796545420042}]"
86,2492191525,11. Demonstrationsdelikte im Kontext rechtsextremer Aufmärsche.,None,"[{'keyword': 'right-wing extremism', 'score': 0.4348615239197596}]"
320,4212796861,Schnelltest detektiert Acinetobacter zuverlässig in Thrombozytenkonzentraten,None,"[{'keyword': 'acinetobacter baumannii', 'score': 0.4922739330281245}]"
1670,3186108928,「個我」與「大我」以雙文化自我觀點建構台灣大學生生涯敘說,None,[]
1029,4243319117,Specification for Copper Alloys in Ingot Form,None,"[{'keyword': 'aluminum alloy', 'score': 0.4852800507574163}]"
2144,4233280353,Cartography and the Mind of Man,None,"[{'keyword': 'crowdsourced mapping', 'score': 0.47493198735342546}]"


### Exploring keywords

In [56]:
def transform_keyword_for_normalization(keyword):

    normalized_keyword = keyword.lower().replace("-", " ").replace(" ", "")

    normalized_keyword = normalized_keyword[:-1] if normalized_keyword.endswith('s') else normalized_keyword
    
    return normalized_keyword

In [103]:
def get_final_keyword_key(display_name):
    return display_name.lower().replace("-", " ").replace("  ", " ").replace(" ", "-")

In [86]:
def get_final_display_name(name_options):
    if len(name_options) == 1:
        return name_options[0]
    else:
        name_len = [len(x) for x in name_options]
        hyphens = [1 if '-' in x else 0 for x in name_options]
        num_caps = [len([x for x in i if x.isupper()]) for i in name_options]

        max_hyphens = 0
        max_name_len = 0
        max_num_caps = 0

        final_name = ""

        for n_len, hyp, n_caps, name_option in zip(name_len, hyphens, num_caps, name_options):
            if (n_len > max_name_len) or (hyp > max_hyphens) or (n_caps > max_num_caps):
                final_name = name_option
                max_hyphens = hyp
                max_name_len = n_len
                max_num_caps = n_caps

        return final_name

In [79]:
all_keywords_data.drop_duplicates('keywords')['norm_keywords'].value_counts().reset_index().head(50)

,norm_keywords,count
0,antiinflammatoryeffect,5
1,environmentalimpact,5
2,culturallandscape,4
3,socialscience,4
4,antioxidant,4
5,indigenouspeople,4
6,infection,4
7,informationsystem,3
8,ionchannel,3
9,executivefunction,3


In [57]:
all_keywords_data['norm_keywords'] = all_keywords_data['keywords'].apply(transform_keyword_for_normalization)

In [74]:
all_keywords_data[all_keywords_data['norm_keywords']=='environmentalimpact'].drop_duplicates('keywords')[['keywords','norm_keywords']]

,keywords,norm_keywords
31,Environmental Impact,environmentalimpact
303,Environmental Impacts,environmentalimpact
844,environmental impact,environmentalimpact
1781,Environmental impact,environmentalimpact
2158,environmental impacts,environmentalimpact


In [89]:
testing_final_display_name = all_keywords_data.drop_duplicates('keywords') \
    .groupby('norm_keywords')['keywords'].agg(list).reset_index()

In [90]:
testing_final_display_name['display_name'] = testing_final_display_name['keywords'].apply(get_final_display_name)

In [92]:
final_display_name = testing_final_display_name.explode('keywords')

In [96]:
final_display_name[final_display_name['display_name']!=final_display_name['keywords']].sample(20)

,norm_keywords,keywords,display_name
13887,macroeconomic,macroeconomics,Macroeconomics
13419,lfunction,L-functions,L-Functions
5855,developmentalperspective,Developmental Perspective,Developmental Perspectives
9461,genetic,Genetic,Genetics
5428,cytokineresponse,Cytokine Response,Cytokine Responses
15984,nanowire,Nanowire,Nanowires
10681,herbalmedicine,Herbal Medicine,Herbal Medicines
5701,democratization,democratization,Democratization
5702,demographicchange,Demographic Change,Demographic Changes
18214,phagocytosi,phagocytosis,Phagocytosis


In [105]:
all_keywords_data.sample(2)

,topic_id,keywords,embedding,norm_keywords
3662,12380,Language Use,"[-0.006248993333429098, 0.008845660835504532, -0.01615484617650509, 0.008175925351679325, -0.029452728107571602, -0.006868328433483839, 0.03864437714219093, -0.0030936303082853556, -0.01212499849498272, -0.037102360278367996, 0.0031116523314267397, -0.0005261495243757963, -0.013991327956318855, -0.01956641487777233, 0.011280806735157967, -0.027716435492038727, -0.018707947805523872, -0.018916543573141098, -0.02811572700738907, -0.017398526892066002, 0.026372473686933517, -0.03686060011386871, 0.056460220366716385, 0.020736893638968468, 0.029440060257911682, 0.017100997269153595, -0.022327957674860954, -0.039698388427495956, 0.03694598376750946, 0.031937126070261, 0.01896175928413868, 0.0009548076195642352, -0.003883135737851262, -0.04341255500912666, -0.027448663488030434, -0.023881709203124046, 0.0010079983621835709, 0.03506425395607948, -0.07612857222557068, 0.025214383378624916, -0.016610121354460716, -0.013217698782682419, -0.026326145976781845, 0.007642971817404032, 0.029276158660650253, -0.02761874906718731, -0.013838553801178932, -0.012769322842359543, -0.03535623103380203, 0.017231522127985954, 0.004302250687032938, 0.01409862656146288, 0.07538896799087524, -0.02690298855304718, 0.020862950012087822, 0.03409605100750923, -0.02214181236922741, 0.012346716597676277, -0.11162769049406052, -0.0059953960590064526, 0.0037148611154407263, -0.023537710309028625, -0.02836988866329193, -0.002456121612340212, 0.001743837259709835, 0.05477321520447731, 0.0077205924317240715, -0.006779720075428486, -0.0137161361053586, -0.033170975744724274, -0.028857827186584473, 0.024321112781763077, -0.00033033653744496405, 0.0018231753492727876, -0.0624212771654129, 0.018395844846963882, 0.0075958059169352055, -0.010724443010985851, 0.005885151214897633, 0.04309176281094551, 0.028405319899320602, 0.018750227987766266, 0.07195410877466202, 0.030102001503109932, 0.006905003450810909, 0.047721017152071, -0.018437175080180168, -0.0216593649238348, -0.011057544499635696, -0.04210004210472107, -0.010156959295272827, -0.02902982197701931, 0.04995148256421089, -0.02243606373667717, -0.005404568277299404, 0.02306358888745308, 0.002156441565603018, 0.03409701958298683, 0.003906681202352047, -0.010732094757258892, ...]",languageuse
3364,13377,Early-warning Signals,"[-0.020272834226489067, 0.023038744926452637, -0.03902260959148407, 0.011662645265460014, -0.03672855347394943, -0.02471904084086418, 0.0163130946457386, 0.048919692635536194, -0.03360806405544281, 0.024144884198904037, 0.019368639215826988, -0.0012810624903067946, -0.01722915656864643, 0.002881556749343872, 0.02621052786707878, -0.09136362373828888, -0.038993965834379196, 0.006396174430847168, -0.03885417804121971, 0.008368096314370632, -0.022387273609638214, -0.003619459690526128, 0.02272907644510269, 0.02469249628484249, 0.02845045179128647, 0.035668663680553436, -0.0008901162655092776, -0.04272786155343056, -0.019253378733992577, 0.04547995701432228, -0.016771430149674416, -0.018706275150179863, 0.004556451924145222, -0.032376550137996674, -0.027554268017411232, -0.008984275162220001, -0.00037852564128115773, 0.010536681860685349, -0.13373348116874695, 0.02858130820095539, -0.01658102683722973, -0.008294988423585892, 0.0150247598066926, -0.04855076223611832, 0.029269693419337273, -0.015940163284540176, -0.029077038168907166, -0.01414341852068901, 0.01960643194615841, -0.05229971930384636, 0.0007223961292766035, -0.013082755729556084, 0.018507380038499832, -0.006726455874741077, 0.013205704279243946, 0.001084921765141189, -0.03286919370293617, -0.002423306228592992, -0.030759602785110474, -0.07029787451028824, -0.04355090856552124, 0.02852787636220455, -0.01836063712835312, -0.020988885313272476, 0.017624223604798317, 0.01251109316945076, 0.021802235394716263, -0.011098591610789299, -0.011547771282494068, -0.025542430579662323, 0.01639663055539131, -0.02388620935380459, -0.038383446633815765, 0.011915106326341629, -0.0711028

In [100]:
new_all_keywords_data = all_keywords_data.merge(final_display_name[['keywords','display_name']].drop_duplicates(), 
                                                how='inner', on='keywords')

In [104]:
new_all_keywords_data['keyword_id'] = new_all_keywords_data['display_name'].apply(get_final_keyword_key)

In [110]:
new_all_keywords_data[['keyword_id','display_name']].drop_duplicates().to_parquet('keywords_for_casey.parquet')

In [107]:
new_all_keywords_data[['topic_id','keyword_id','display_name','keywords']].sample(20)

,topic_id,keyword_id,display_name,keywords
26810,12712,fascial-plasticity,Fascial Plasticity,Fascial Plasticity
24466,14344,dopamine-agonists,Dopamine Agonists,Dopamine Agonists
4401,10062,metastasis,Metastasis,Metastasis
39428,11115,morphology-based-filters,Morphology-based Filters,Morphology-based Filters
27554,10784,sensory-feedback,Sensory Feedback,Sensory Feedback
32059,11707,eye-tracking,Eye Tracking,Eye Tracking
35498,13825,immigration-policies,Immigration Policies,Immigration Policies
43383,10221,shear-deformation-theory,Shear Deformation Theory,Shear Deformation Theory
23879,12056,approximate-bayesian-computation,Approximate Bayesian Computation,Approximate Bayesian Computation
10192,10546,quality-assurance,Quality Assurance,Quality Assurance


In [106]:
new_all_keywords_data[['topic_id','keyword_id','display_name','keywords','embedding']] \
    .to_parquet('latest_keywords_file.parquet')

In [113]:
new_file = pd.read_parquet("latest_keywords_file.parquet")

In [115]:
new_file.sample(1)

,topic_id,keyword_id,display_name,keywords,embedding
19571,13955,entropy-weight,Entropy Weight,Entropy Weight,"[0.0026810334529727697, 0.004703668877482414, -0.007654732093214989, -0.008646409027278423, -0.03681402653455734, -0.07908197492361069, 0.0198382381349802, -0.002426604274660349, -0.0023849685676395893, -0.017848581075668335, 0.012804345227777958, 0.008585539646446705, -0.009734250605106354, -0.003086685435846448, 0.030598612502217293, -0.01647077314555645, -0.015609859488904476, -0.015078515745699406, -0.06924502551555634, -0.00593704404309392, -0.01282799057662487, -0.023465177044272423, 0.009105665609240532, 0.01413060538470745, 0.005108551122248173, 0.05292129889130592, -0.020420623943209648, 0.010411563329398632, 0.017799820750951767, -0.006627588998526335, 0.028011299669742584, 0.0054331207647919655, 0.009877088479697704, -0.05361875519156456, -0.010135981254279613, -0.021687058731913567, 0.03789178282022476, -0.01798289082944393, -0.05849285423755646, 0.010320303030312061, 0.02150595933198929, -0.0587431937456131, -0.03318381682038307, -0.055787794291973114, 0.04692681506276131, -0.04903823882341385, 0.004611814394593239, -0.04053961858153343, -0.050500571727752686, -0.025555351749062538, -0.02603253908455372, -0.003764048218727112, 0.03116334043443203, 0.013714722357690334, -0.0062173279002308846, 0.029825085774064064, -0.05439801514148712, -0.02367313764989376, -0.062439143657684326, -0.06306125223636627, -0.020475858822464943, 0.02240213379263878, 0.012150653637945652, 0.023572096601128578, 0.0149641502648592, 0.01923186145722866, -0.016474340111017227, 0.026129042729735374, 0.004714925307780504, 0.021808037534356117, -0.00037086880183778703, -0.022293612360954285, -0.029507363215088844, 0.02769860066473484, -0.06216927617788315, 0.017519567161798477, -0.03485232964158058, -0.030676495283842087, 0.0017207617638632655, 0.023816987872123718, 0.08586268126964569, 0.004299593158066273, -0.017042577266693115, 0.030130436643958092, -0.0073374370113015175, 0.07074105739593506, -0.032510049641132355, -0.056233249604701996, -0.03915371000766754, -0.06902778148651123, -0.028515709564089775, 0.01412420067936182, -0.0016620381502434611, 0.0010150232119485736, -0.005887164734303951, 0.01973532885313034, -0.03804684057831764, 0.015048989094793797, 0.023880621418356895, 0.006462787743657827, ...]"


In [125]:
new_file[['keywords','keyword_id']].drop_duplicates(subset=['keywords']).set_index('keywords').to_dict()['keyword_id']\
['Imaging']

'imaging'

In [111]:
curr_file = pd.read_parquet("./003_Deployment/model_to_api/container/model_artifacts/model/keywords_latest.parquet")